In [13]:
from tqdm import tqdm_notebook
from itertools import product
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch._C import device
import utils
import eval_methods
import modules
import time
device = torch.device(f"cuda:{0}" if torch.cuda.is_available() else "cpu")


## Dataset

In [3]:
filepath = 'https://raw.githubusercontent.com/selva86/datasets/master/Raotbl6.csv'
macro_data = pd.read_csv(filepath, parse_dates=['date'], index_col='date')
print(macro_data.shape)  # (123, 8)
macro_data.head()

(123, 8)


,rgnp,pgnp,ulc,gdfco,gdf,gdfim,gdfcf,gdfce
date,,,,,,,,
1959-01-01,1606.4,1608.3,47.5,36.9,37.4,26.9,32.3,23.1
1959-04-01,1637.0,1622.2,47.5,37.4,37.5,27.0,32.2,23.4
1959-07-01,1629.5,1636.2,48.7,37.6,37.6,27.1,32.4,23.4
1959-10-01,1643.4,1650.3,48.8,37.7,37.8,27.1,32.5,23.8
1960-01-01,1671.6,1664.6,49.1,37.8,37.8,27.2,32.4,23.8


In [4]:
date=pd.to_datetime(macro_data.index)

In [5]:
train,test = train_test_split(macro_data, test_size=0.2, random_state=42)

In [6]:
train.shape

(98, 8)

In [7]:
test.shape

(25, 8)

In [8]:
date_train=date[:98]
date_test=date[98:]

In [9]:
date_train.shape

(98,)

In [10]:
date_test.shape

(25,)

## Encode the timstamps

In [14]:
train_encoded_input = modules.temporal_encoding(date_train)
test_encoded_input=modules.temporal_encoding(date_test)

               y   m  d  h  min  s
date                              
1959-01-01  1959   1  1  0    0  0
1959-04-01  1959   4  1  0    0  0
1959-07-01  1959   7  1  0    0  0
1959-10-01  1959  10  1  0    0  0
1960-01-01  1960   1  1  0    0  0
...          ...  .. .. ..  ... ..
1982-04-01  1982   4  1  0    0  0
1982-07-01  1982   7  1  0    0  0
1982-10-01  1982  10  1  0    0  0
1983-01-01  1983   1  1  0    0  0
1983-04-01  1983   4  1  0    0  0

[98 rows x 6 columns]
               y   m  d  h  min  s
date                              
1983-07-01  1983   7  1  0    0  0
1983-10-01  1983  10  1  0    0  0
1984-01-01  1984   1  1  0    0  0
1984-04-01  1984   4  1  0    0  0
1984-07-01  1984   7  1  0    0  0
1984-10-01  1984  10  1  0    0  0
1985-01-01  1985   1  1  0    0  0
1985-04-01  1985   4  1  0    0  0
1985-07-01  1985   7  1  0    0  0
1985-10-01  1985  10  1  0    0  0
1986-01-01  1986   1  1  0    0  0
1986-04-01  1986   4  1  0    0  0
1986-07-01  1986   7  1  0    0 

In [15]:
train=train.values
test=test.values

## Train the model on the train dataset

In [17]:
hidden_dim=256
batch_size=32 # 32)
epochs=1 # For simplicity, we set it as 1, however originally we set it as 10000
earlystopping_patience=30
first_omega_0=3000

# Model initialization

model = modules.Siren(
    in_features=train_encoded_input.shape[1],
    out_features=train.shape[1],
    hidden_features=hidden_dim,
    hidden_layers=3,
    first_omega_0=first_omega_0,
    outermost_linear=True,
)
model.to(device)

optim = torch.optim.Adam(lr=1e-4, params=model.parameters())

In [18]:
data_train = modules.Timedata(train, train_encoded_input)
train_dataloader = DataLoader(
    data_train,
    shuffle=True,
    batch_size=batch_size,
    pin_memory=True,
    num_workers=0,
)

early_stopping = utils.EarlyStopping(
    patience=earlystopping_patience, verbose=False
)

epoch_time = []
for step in range(epochs):
    epoch_start = time.time()
    model_loss = 0
    for batch_model_input, batch_ground_truth in train_dataloader:
        batch_model_input = batch_model_input.to(device)
        batch_ground_truth = batch_ground_truth.to(device)

        batch_model_output, _ = model(batch_model_input)
        loss = F.mse_loss(batch_model_output, batch_ground_truth)
        optim.zero_grad()
        loss.backward()
        optim.step()
        model_loss += loss.item()
        batch_model_input = batch_model_input.detach().cpu()
        batch_ground_truth = batch_ground_truth.detach().cpu()
    epoch_time.append(time.time() - epoch_start)
    early_stopping(model_loss)
    if early_stopping.early_stop:
        break

print("average training time per epoch: ", np.mean(epoch_time))

average training time per epoch:  0.14271235466003418


## Retrain the model on the test set

In [19]:
data_test = modules.Timedata(test, test_encoded_input)
test_dataloader = DataLoader(
    data_test,
    shuffle=True,
    batch_size=batch_size,
    pin_memory=True,
    num_workers=0,
)

early_stopping = utils.EarlyStopping(
    patience=earlystopping_patience, verbose=False
)

print("re-training start")
for step in range(epochs):
    epoch_start = time.time()
    model_loss = 0
    for batch_model_input, batch_ground_truth in train_dataloader:
        batch_model_input = batch_model_input.to(device)
        batch_ground_truth = batch_ground_truth.to(device)
        batch_model_output, _ = model(batch_model_input)
        loss = F.mse_loss(batch_model_output, batch_ground_truth)
        optim.zero_grad()
        loss.backward()
        optim.step()
        model_loss += loss.item()
        batch_model_input = batch_model_input.detach().cpu()
        batch_ground_truth = batch_ground_truth.detach().cpu()
    early_stopping(model_loss)
    if early_stopping.early_stop:
        break
print("re-training end")

re-training start
re-training end


In [20]:
total_input = data_test.timepoints
model = model.cpu()
total_ground_truth = data_test.data_ready
total_model_output, _ = model(total_input)

anomaly_score = np.mean(
    np.abs(
        np.squeeze(
            total_ground_truth.numpy()
            - total_model_output.detach().cpu().numpy()
        )
    ),
    axis=1,
)


### Threshold

to find the nest threshold we us the function eval_methods.bf_search to search for the best F1 score corresponding to the best threshold.but the data need to be labeled and I did,'t find any labeled dataset

In [ ]:
accuracy, threshold = eval_methods.bf_search(anomaly_score, y_test, verbose = False)
print("Precision: {}, Recall {}, F1-score: {}".format(accuracy[1], accuracy[2], accuracy[0]))

(25,)